## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Hello There In this notebokk we are going to scrape the Full time and part time jobs data for the data related jobs like:<br>
Data analyst, ML dev, Data scientist, Data engineer etc ..<br><br>

**Important note:**
We will analyze some countries only for indeed becuase indeed needs to specify which country to look at.

## <center><strong>Importing <span style= 'color: #48e0dc'>Packeges

In [105]:
import time
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from IPython.display import set_matplotlib_formats
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [3]:
COLORS = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']

%matplotlib inline

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']
MAX_PAGES = 300

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings('ignore')

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_9984\725494067.py:9: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center><strong>Setting up the<span style= 'color: #5cd3f7'> web scrapers

In [4]:
data_jobs_titles = ['Data entry', 'Data engineer']
                    # 'Data scientist', 'Data analyst',
                    # 'ML devoloper']

indeed_countries = ['uk', 'de']#, 'www'] USA don't have to be spesfied in the URL so we will use 'www' instead
linkedin_countries = ['European Union', 'United States']

In [5]:
def scrape_page(url: str,
                ip_and_port: tuple[str]= None,
                retrieve_new_url= None) -> BeautifulSoup:

    chrome_options = Options()

    if ip_and_port:
        chrome_options.add_argument(f'--proxy-server={ip_and_port[0]}:{ip_and_port[1]}')
        webdriver_service = Service(GeckoDriverManager().install())
        driver = webdriver.Chrome(service= webdriver_service, options= chrome_options)

    else:
        driver = webdriver.Chrome()

    driver.get(url)
    
    wait = WebDriverWait(driver, 10)
    # Wait until the page is fully loaded
    time.sleep(1)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    html = driver.page_source

    if retrieve_new_url:

        new_url = driver.current_url
        driver.quit()

        return BeautifulSoup(html), new_url

    driver.quit()

    return BeautifulSoup(html)

In [6]:
def indeed_scraper(page: int,
                   country: str,
                   job_title: str) -> BeautifulSoup:

    modefid_page = (page - 1) * 10

    data_job_title = job_title.replace(' ', '+')

    # Pages in indeed.com are zero based indexed
    url: str = f'https://{country}.indeed.com/jobs?q={data_job_title}&sc=0kf%3Aattr%28DSQF7%29%3B&start={modefid_page}' + \
    '&pp=gQClAAAAAAAAAAAAAAACCXklWwCSAQIBK7YBBwDUbujiRGBahYW5TppjdUz7DjXn3aPZSbT47IoJ5LLbuzpYcXwZdzJ6rKHf' + \
    '6gPWFkXVGxKKGxW-JAKb8BFo_hZAkBd7trBBTY32J2CrOuA3V9dGD_bre-lArmi9DRYlcah6hvoRfsYUNYSoQwIa8VOMZMxvH-s2Dlh' + \
    'UPvUP-_Dz9ls4i-OLqVGpGh4AAA&vjk=b997bb0dddadea'

    soup = scrape_page(url)
    loaded_page = int(soup.find('button', 'css-ns2mzi e8ju0x51').text)

    # available_pages = []

    # for button in soup.find_all('button', 'css-1qt7hdn e8ju0x50'):
    #     available_pages.append(int(button.text))

    if (int(page) != loaded_page):
        # We will use KeyError as standard to represent getting out of the max pages.
        raise KeyError
        
    print(f'Page {page} Loaded successfully.')

    return  soup

In [7]:
def check_for_content(driver):
    
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    try:
        driver.find_element(By.CLASS_NAME, 'loader loader--show')
        continue_ = True

    except:
        try:
            driver.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button infinite-scroller__show-more-button--visible')
            continue_ = True

        except:
            continue_ = False

    return  continue_

In [8]:
def linkedin_scraper(country: str, # This scraper scrapes all pages at once
                     job_title: str) -> BeautifulSoup:

    # I know that code logic is wrong but at least it's running completly fine.

    driver = webdriver.Chrome()
    url: str = f'https://www.linkedin.com/jobs/search/?keywords={job_title}&' + \
                f'location={country}&refresh=true&start=0&position=1&pageNum=0'

    driver.get(url)

    while True:
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

            wait = WebDriverWait(driver, 10)
            wait.until(lambda driver: check_for_content(driver))
            # print(is_element_present(driver))

        except:
            break

    soup = BeautifulSoup(driver.page_source)
    driver.quit()

    return  soup

## <center><strong><span style= 'color: #4895e0'>Collecting</span> jobs data

**Now we will need for three steps to collect the data we need:**

1. Collect all pages we need from each scraper and stack them together as HTML code.
2. Scrape for jobs cards links from linkedin and indeed.
3. Scrape for the data in those links and store the data in a DataFrame then in SQLite
4. We may also consider saving total results

#### **Stack pages together as HTML code**

In [9]:
soups = {'soup': [],
         'job_title': [],
         'country': [],
         'platform': []}

flag: bool = False
max_page: int = 7

for country in indeed_countries:
    for job_title in data_jobs_titles:
        stacked_pages_soup: str = ''

        for i in count(0):

            try:
                page = i + 1
                soup: BeautifulSoup = indeed_scraper(country= country,
                                                     job_title= job_title,
                                                     page= page)
                stacked_pages_soup += '\n<br> ' + str(soup)
                if page > max_page:
                    print('Finished loading current country or job title.\n')
                    break
            
            except KeyError as e:
                print('Finished loading current country or job title.\n')
                break

        soups['platform'].append('Indeed')
        soups['soup'].append(stacked_pages_soup)
        soups['job_title'].append(job_title)
        soups['country'].append(country)

for country in linkedin_countries:
    for job_title in data_jobs_titles:

        soup: BeautifulSoup = linkedin_scraper(country= country,
                                job_title= job_title)

        soups['platform'].append('LinkedIn')
        soups['soup'].append(soup)
        soups['job_title'].append(job_title)
        soups['country'].append(country)

Page 1 Loaded successfully.
Page 2 Loaded successfully.
Page 3 Loaded successfully.
Page 4 Loaded successfully.
Page 5 Loaded successfully.
Page 6 Loaded successfully.
Page 7 Loaded successfully.
Page 8 Loaded successfully.
Finished loading current country or job title.

Page 1 Loaded successfully.
Page 2 Loaded successfully.
Page 3 Loaded successfully.
Page 4 Loaded successfully.
Page 5 Loaded successfully.
Page 6 Loaded successfully.
Page 7 Loaded successfully.
Page 8 Loaded successfully.
Finished loading current country or job title.

Page 1 Loaded successfully.
Page 2 Loaded successfully.
Finished loading current country or job title.

Page 1 Loaded successfully.
Page 2 Loaded successfully.
Page 3 Loaded successfully.
Page 4 Loaded successfully.
Page 5 Loaded successfully.
Page 6 Loaded successfully.
Page 7 Loaded successfully.
Page 8 Loaded successfully.
Finished loading current country or job title.



In [11]:
pd.DataFrame(soups).head(6)

,soup,job_title,country,platform
0,"\n<br> <html class=""js-focus-visible"" data-js-...",Data entry,uk,Indeed
1,"\n<br> <html class=""js-focus-visible"" data-js-...",Data engineer,uk,Indeed
2,"\n<br> <html class=""js-focus-visible"" data-js-...",Data entry,de,Indeed
3,"\n<br> <html class=""js-focus-visible"" data-js-...",Data engineer,de,Indeed
4,"[[[\n, <meta content=""d_jobs_guest_search"" nam...",Data entry,European Union,LinkedIn
5,"[[[\n, <meta content=""d_jobs_guest_search"" nam...",Data engineer,European Union,LinkedIn


#### **Collecting jobs links**

In [81]:
soups = pd.DataFrame(soups) # I will convert the soups dict to a df so we can do masking easly.

indeed_jobs_links: set = set()
indeed_companies_links: set = set()

# linkedin_jobs_links: set = set()
# linkedin_companies_links: set = set()

for i, soup in enumrate(soups[soups['platform'] == 'Indeed']['soup']):
    soup = BeautifulSoup(soup)
    
    for job_id in soup.find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
        job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job_id['data-jk'])
        
        indeed_jobs_links.add(job_link)
        indeed_companies_links.add(scrape_page(job_link).find('a', 'css-775knl emf9s7v0')['href'])

    soups['jobs_links'][i] = indeed_jobs_links
    soups['companies_links'][i] = indeed_companies_links

In [82]:
indeed_jobs_links

{'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=00a2d82c2102d54c',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=022400aaf3216e62',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=0242b3b537cbfe21',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=02c7f366dbe43947',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=047e9a55c7ff34f0',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=04f5a757aeb46b73',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=05579eee5a65c66a',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=062d60b0f8541708',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=06c8ae2e778845ac',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=08dbe545f0f71b73',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=098bd8b6268a68fd',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=0b089e89201b0e67',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=0b72e47e12733dbc',
 'https://ww

In [63]:
test_html = scrape_page('https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=26c4d3754f443acb')


In [73]:
test_html.find_all('div','jobsearch-JobMetadataFooter')

[<div class="jobsearch-JobMetadataFooter"><div class="mosaic mosaic-provider-reportcontent" id="mosaic-provider-reportcontent" style="--mosaic-placeholder-width: 0; --mosaic-placeholder-height: 0;"></div></div>]

In [104]:
scrape_page('https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=00a2d82c2102d54c')

<html class="js-focus-visible" data-js-focus-visible="" dir="ltr" lang="en"><head>
<link href="/images/favicon.ico" rel="shortcut icon"/>
<title>D Jobs, Employment | Indeed.com</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="191,029 D jobs available on Indeed.com. Apply to Customer Service Representative, Program Associate, Learning and Development Associate and more!" name="description"/>
<meta content="D Jobs, Employment,  careers,  employment,  job listings,  job search,  search engine, work in " name="keywords"/>
<meta content="origin-when-cross-origin" name="referrer"/>
<link href="https://www.indeed.com/q-d-jobs.html" rel="canonical"/>
<link href="android-app://com.indeed.android.jobsearch/https/www.indeed.com/m/jobs?q=d&amp;l=&amp;from=searchOnHP&amp;vjk=00a2d82c2102d54c" rel="alternate"/>
<link href="ios-app://https/www.indeed.com/m/jobs?q=d&amp;l=&amp;from=searchOnHP&amp;vjk=00a2d82c2102d54c" rel="alternate"/>
<link href="https://rss